# SED for Trappist-1
This notebook executes the lines of code presented in Filippazzo et al. (submitted to PASP). Below I generate and analyze an SED for the M8 dwarfs Trappist-1 and Gl 752B. Then I create a catalog of SEDs for analysis.

In [33]:
# Imports
from astropy import units as q
from bokeh.io import output_notebook
from bokeh.plotting import show
from pkg_resources import resource_filename
output_notebook()

Loading BokehJS ...

## Initialization
Import the `SED` class and create a Trappist-1 object.

In [34]:
from sedkit import SED
trap1 = SED(name='Trappist-1')

Setting parallax to (<Quantity 80.4512 mas>, <Quantity 0.1211 mas>) and distance to (<Quantity 12.43 pc>, <Quantity 0.02 pc>, <Quantity 0. pc>).
Setting radius to (<Quantity 0.12250478 solRad>, <Quantity 0.00153489 solRad>)
Setting spectral_type to (68.0, 0.5, 'V', None, None).
1 record for Trappist-1 found in Simbad.
Setting name to Trappist-1 and sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (346.62236833, -5.0413975)>
Setting age to (<Quantity 6. Gyr>, <Quantity 4. Gyr>)


## Photometry
Add photometry manually...

In [35]:
trap1.add_photometry('Johnson.V', 18.798, 0.082, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.R', 16.466, 0.065, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.I', 14.024, 0.115, ref='Costa et al. (2006)')

...and retrieve some from Simbad.

In [36]:
trap1.find_2MASS()
trap1.find_WISE()

1 record found in 2MASS.
1 record found in WISE.


In [37]:
# Print the photometry table
trap1.photometry[['band', 'eff', 'app_magnitude', 'app_magnitude_unc', 'ref']].pprint()

   band     eff   app_magnitude app_magnitude_unc         ref        
             um                                                      
--------- ------- ------------- ----------------- -------------------
Johnson.V 0.54661          18.8             0.082 Costa et al. (2006)
Cousins.R  0.6358         16.47             0.065 Costa et al. (2006)
Cousins.I 0.78292         14.02             0.115 Costa et al. (2006)
  2MASS.J   1.235         11.35             0.022          II/246/out
  2MASS.H   1.662         10.72             0.021          II/246/out
 2MASS.Ks   2.159          10.3             0.023          II/246/out
  WISE.W1  3.3526         10.07             0.023      II/328/allwise
  WISE.W2  4.6028         9.805              0.02      II/328/allwise
  WISE.W3 11.5608          9.51             0.044      II/328/allwise
  WISE.W4 22.0883          8.56               nan      II/328/allwise


## Spectra

In [38]:
# Grab spectrum files
nir_spec = resource_filename('sedkit', 'data/Trappist-1_NIR.fits')
opt_spec = resource_filename('sedkit', 'data/Trappist-1_OPT.fits')

# Add the spectra
trap1.add_spectrum_file(nir_spec, name='NIR spectrum', wave_units=q.um, flux_units=q.W/q.m**2/q.um, ref='Filippazzo et al. (2015)')
trap1.add_spectrum_file(opt_spec, name='OPT spectrum', wave_units=q.AA, flux_units=q.erg/q.s/q.cm**2/q.AA, trim=[(0.9*q.um, 2*q.um)], ref='Cruz et al. (2003)')

# Print the table
trap1.spectra[['name', 'wave_min', 'wave_max', 'resolution', 'ref']].pprint()

    name     wave_min wave_max resolution           ref           
                um       um                                       
------------ -------- -------- ---------- ------------------------
NIR spectrum    0.653    2.566        563 Filippazzo et al. (2015)
OPT spectrum      0.6      0.9       1034       Cruz et al. (2003)


## Other Data
Set the spectral type.

In [39]:
trap1.spectral_type = 'M8'

Setting spectral_type to (68.0, 0.5, 'V', None, None).


Set the parallax.

In [40]:
trap1.parallax = 80.4512*q.mas, 0.1211*q.mas

Setting parallax to (<Quantity 80.4512 mas>, <Quantity 0.1211 mas>) and distance to (<Quantity 12.43 pc>, <Quantity 0.02 pc>, <Quantity 0. pc>).


Set the age.

In [41]:
trap1.age = 7.6*q.Gyr, 2.2*q.Gyr

Setting age to (<Quantity 7.6 Gyr>, <Quantity 2.2 Gyr>)


## Build SED

In [42]:
trap1.results.pprint(max_lines=-1)

     param               value                  unc               units    
---------------- --------------------- ---------------------- -------------
            Lbol              2.14e+30               1.41e+28       erg / s
        Lbol_sun                 -3.25                  0.003            --
            Mbol                12.886                  0.006            --
             SpT                   M8V                     --            --
         SpT_fit                    --                     --            --
            Teff                  2536                     16             K
         Teff_bb                    --                     --            --
        Teff_evo                2640.4      5.300000000000182             K
             age                   7.6                    2.2           Gyr
       bb_source                    --                     --            --
       blackbody                    --                     --            --
            

## Fit the data

In [43]:
# Fit to infer spectral type
trap1.fit_spectral_type()

Best fit:  Opt:M8V


In [44]:
# Fit to infer fundamental params
from sedkit import BTSettl
trap1.fit_modelgrid(BTSettl())

Best fit:  0.0/2700.0/5.5/0.0


## Mass, surface gravity, and Teff
Set the evolutionary model grid and infer fundamental parameters.

In [45]:
trap1.evo_model = 'DUSTY00'

In [46]:
trap1.teff_from_age(plot=True)

In [47]:
trap1.logg_from_age(plot=True)

In [48]:
trap1.mass_from_age(plot=True, mass_units=q.Mjup)

## Results

In [49]:
# Table of the results
trap1.results.pprint(max_lines=-1)

     param               value                  unc               units    
---------------- --------------------- ---------------------- -------------
            Lbol              2.14e+30               1.41e+28       erg / s
        Lbol_sun                 -3.25                  0.003            --
            Mbol                12.886                  0.006            --
             SpT                   M8V                     --            --
         SpT_fit               Opt:M8V                     --            --
            Teff                  2536                     16             K
         Teff_bb                    --                     --            --
        Teff_evo                2640.4      5.300000000000182             K
             age                   7.6                    2.2           Gyr
       bb_source                    --                     --            --
       blackbody                    --                     --            --
            

In [50]:
trap1.plot(integral=True)

Figure(id='6827', ...)

In [51]:
trap1.refs

{'parallax': '2018yCat.1345....0G',
 'photometry': {'2MASS.H': 'II/246/out',
  '2MASS.J': 'II/246/out',
  '2MASS.Ks': 'II/246/out',
  'Cousins.I': 'Costa et al. (2006)',
  'Cousins.R': 'Costa et al. (2006)',
  'Johnson.V': 'Costa et al. (2006)',
  'WISE.W1': 'II/328/allwise',
  'WISE.W2': 'II/328/allwise',
  'WISE.W3': 'II/328/allwise',
  'WISE.W4': 'II/328/allwise'},
 'spectra': {'NIR spectrum': 'Filippazzo et al. (2015)',
  'OPT spectrum': 'Cruz et al. (2003)'},
 'spectral_type': '2007AJ....133.2258S'}

### SED for Gl 752B
Now for comparison, we'll make an SED for the M dwarf Gl 752B all in one block of code.

In [52]:
# Initialize the object
gl = SED('Gl 752B', method_list=['find_2MASS'])

# Specify paramaters
gl.radius = 0.102*q.R_sun, 0.005*q.R_sun
gl.age = 1.0*q.Gyr, 0.1*q.Gyr

# Grab spectrum files
nir_spec = resource_filename('sedkit', 'data/Gl752B_NIR.fits')
opt_spec = resource_filename('sedkit', 'data/Gl752B_OPT.txt')

# Add the spectra
gl.add_spectrum_file(opt_spec, snr=10, wave_units=q.AA, flux_units=q.erg/q.s/q.cm**2/q.AA)
gl.add_spectrum_file(nir_spec, wave_units=q.um, flux_units=q.erg/q.s/q.cm**2/q.AA)

# Fit 
gl.fit_spectral_type()
gl.fit_modelgrid(BTSettl())

# Infer fundamental parameters
gl.teff_from_age()
gl.logg_from_age()
gl.mass_from_age(mass_units=q.Mjup)

# Plot it
gl.plot(integral=True)

Setting parallax to (<Quantity 168.962 mas>, <Quantity 0.1299 mas>) and distance to (<Quantity 5.92 pc>, <Quantity 0. pc>, <Quantity 0. pc>).
Setting radius to (<Quantity 0.12250478 solRad>, <Quantity 0.00153489 solRad>)
Setting spectral_type to (68.0, 0.5, 'V', None, None).
1 record for Gl 752B found in Simbad.
Setting name to Gl 752B and sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (289.24004292, 5.15044111)>
Setting age to (<Quantity 6. Gyr>, <Quantity 4. Gyr>)
1 record found in 2MASS.
Setting radius to (<Quantity 0.102 solRad>, <Quantity 0.005 solRad>)
Setting age to (<Quantity 1. Gyr>, <Quantity 0.1 Gyr>)
Best fit:  Opt:M7V
Best fit:  0.0/2600.0/5.5/0.0


Figure(id='7470', ...)

And just as a double check, our derived Teff of 2691K agrees with Linsky et al. (1995).

# Catalog of M Dwarfs
Initialize a `Catalog` object and add the Trappist-1 and Gl 752B SEDs.

In [53]:
from sedkit import Catalog
cat = Catalog('M Dwarfs')
cat.add_SED(trap1)
cat.add_SED(gl)
cat.results

Successfully added SED 'Trappist-1'
Successfully added SED 'Gl 752B'


name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,Johnson.V,Johnson.V_unc,M_Johnson.V,M_Johnson.V_unc,Cousins.R,Cousins.R_unc,M_Cousins.R,M_Cousins.R_unc,Cousins.I,Cousins.I_unc,M_Cousins.I,M_Cousins.I_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc,WISE.W4,WISE.W4_unc,M_WISE.W4,M_WISE.W4_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
Trappist-1,7.6,2.2,12.43,0.02,80.4512,0.1211,0.12250477577816277,0.0015348931895269313,68.0,0.5,M8V,Opt:M8V,None,0.0010283516021445394,1.1587359896461991e-10,6.626838504032113e-13,13.358,0.006,2.1420899707212794e+30,1.405689152659424e+28,-3.252,0.003,12.886,0.006,5.285607017543859,0.0012421052631577112,0.09109824561403508,0.00031052631578951106,2536.0,16.0,2640.4,5.300000000000182,None,<sedkit.sed.SED object at 0x132dcfe48>,18.8,0.082,18.34,0.082,16.47,0.065,16.0,0.065,14.02,0.115,13.55,0.115,11.35,0.022,10.875,0.022,10.72,0.021,10.25,0.021,10.3,0.023,9.83,0.023,10.07,0.023,9.59,0.023,9.805,0.02,9.33,0.02,9.51,0.044,9.03,0.044,8.56,nan,8.09,nan
Gl 752B,1.0,0.1,5.92,0.0,168.962,0.1298999935388565,0.102,0.005,68.0,0.5,M8V,Opt:M7V,None,0.0002411319874227047,4.4901623109765133e-10,2.7433476323814696e-12,11.887,0.007,1.8828529716609967e+30,1.150363814087884e+28,-3.308,0.003,13.025,0.007,5.2899072543617995,0.03390092847668491,0.08863269054178145,0.0037715539230690764,2691.0,66.0,2590.7257116620754,27.706978879706185,None,<sedkit.sed.SED object at 0x1215c8c50>,--,--,--,--,--,--,--,--,--,--,--,--,9.91,0.025,11.05,0.025,9.23,0.026,10.37,0.026,8.766,0.022,9.91,0.022,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


Plot the two stars, normlized to H band.

In [59]:
show(cat.plot_SEDs(['Trappist-1', 'Gl 752B'], normalize=[(1.4, 1.8)], best_fit=False))

## Catalog from a list
Now let's add another 23 M dwarf SEDs.

In [60]:
# Add the sources from file
cat.from_file(resource_filename('sedkit', 'data/sources.txt'))

Generating SEDs for 23 sources from /Users/jfilippazzo/Documents/Modules/sedkit/sedkit/data/sources.txt
Successfully added SED 'LHS 102'
Successfully added SED 'LP 704-43'
Successfully added SED 'LHS 1'
Successfully added SED 'LHS 2'
Successfully added SED 'LHS 1047AB'
Successfully added SED 'LHS 3'
Successfully added SED 'LP 944-20'
Successfully added SED 'LHS 2065'
Successfully added SED 'LHS 1070 ABC'
Successfully added SED 'LHS 1075'
Successfully added SED 'LP 465-27'
Successfully added SED 'LHS 114 AB'
Successfully added SED 'LHS 115'
Successfully added SED 'LP 405-39'
Successfully added SED 'LP 406-29'
Successfully added SED 'LHS 131'
Successfully added SED 'LHS 1179'
Successfully added SED 'LHS 132'
Successfully added SED 'LHS 1183'
Successfully added SED 'LHS 134'
Successfully added SED 'LHS 1208'
Successfully added SED 'LP 467-16'
Successfully added SED 'LHS 138'


And here are some plots!

In [64]:
col_mag = cat.plot('spectral_type', 'Lbol_sun', legend='M dwarfs', order=2, identify=['Trappist-1'])
show(col_mag)

Successfully added SED 'Trappist-1'


In [65]:
BC_J = cat.plot('2MASS.J-2MASS.H', 'M_2MASS.J', legend='M dwarfs', identify=['Trappist-1'])
show(BC_J)

Successfully added SED 'Trappist-1'
